In [2]:
import torch
import torch.nn as nn

@torch.no_grad()
def init_last_layer_increasing(module: nn.Module, start: float = -2.0, end: float = 2.0):
    """
    Initialize the weights of the last nn.Linear in `module` so that the outputs
    are linearly increasing between [start, end].

    Assumes the last layer is nn.Linear with shape (n_thetas, in_features).
    """
    # Find last Linear layer
    last_linear = None
    for m in reversed(list(module.modules())):
        if isinstance(m, nn.Linear):
            last_linear = m
            break
    if last_linear is None:
        raise ValueError("No nn.Linear layer found in module.")

    n_thetas = last_linear.out_features
    in_features = last_linear.in_features

    # Desired increasing values
    values = torch.linspace(start, end, steps=n_thetas)

    # If in_features > 1, just repeat values / distribute across input dims
    w = torch.zeros((n_thetas, in_features))
    w[:, 0] = values  # put increasing sequence in first input channel

    last_linear.weight.copy_(w)

    if last_linear.bias is not None:
        last_linear.bias.zero_()  # optional: reset bias to 0

    return last_linear


class SimpleIntercept(nn.Module):
    """
    Intercept term , hI()
    Attributes:
        n_thetas (int): how many output thetas, for ordinal target this is the number of classes - 1, thetas are order of bernsteinpol() in continous case
    """
    def __init__(self, n_thetas=20):
        super(SimpleIntercept, self).__init__()  
        self.fc = nn.Linear(1,n_thetas, bias=False)

    def forward(self, x):
        return self.fc(x)
    

In [2]:
simple = SimpleIntercept(n_thetas=20)

last_layer = init_last_layer_increasing(simple, start=-3.0, end=3.0)

x = torch.ones(1, 1)
out = simple(x)

print("Weights:", last_layer.weight.squeeze())
print("Output :", out.squeeze())


Weights: tensor([-3.0000, -2.6842, -2.3684, -2.0526, -1.7368, -1.4211, -1.1053, -0.7895,
        -0.4737, -0.1579,  0.1579,  0.4737,  0.7895,  1.1053,  1.4211,  1.7368,
         2.0526,  2.3684,  2.6842,  3.0000], grad_fn=<SqueezeBackward0>)
Output : tensor([-3.0000, -2.6842, -2.3684, -2.0526, -1.7368, -1.4211, -1.1053, -0.7895,
        -0.4737, -0.1579,  0.1579,  0.4737,  0.7895,  1.1053,  1.4211,  1.7368,
         2.0526,  2.3684,  2.6842,  3.0000], grad_fn=<SqueezeBackward0>)


In [8]:
class ComplexInterceptDefaultTabular(nn.Module):
    """
    Complex shift term for tabular data. Can be any neural network architecture
    Attributes:
        n_thetas (int): number of features/predictors
    """
    def __init__(self, n_features=1,n_thetas=20):
        super(ComplexInterceptDefaultTabular, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(n_features, 8)  # First hidden layer (X_i -> 8)
        self.relu1 = nn.ReLU()               # ReLU activation
        self.fc2 = nn.Linear(8, 8)           # Second hidden layer (8 -> 8)
        self.relu2 = nn.ReLU()               # ReLU activation
        self.fc3 = nn.Linear(8, n_thetas, bias=False)  # Output layer (8 -> n_thetas, no bias)
        
    def forward(self, x):
        # Forward pass through the network
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [9]:
complex = ComplexInterceptDefaultTabular(n_thetas=20)

last_layer = init_last_layer_increasing(simple, start=-1.0, end=1.0)

x = torch.ones(1, 1)
out = complex(x)

print("Weights:", last_layer.weight.squeeze())
print("Output :", out.squeeze())

Weights: tensor([-1.0000, -0.8947, -0.7895, -0.6842, -0.5789, -0.4737, -0.3684, -0.2632,
        -0.1579, -0.0526,  0.0526,  0.1579,  0.2632,  0.3684,  0.4737,  0.5789,
         0.6842,  0.7895,  0.8947,  1.0000], grad_fn=<SqueezeBackward0>)
Output : tensor([ 0.0335,  0.0188, -0.0706,  0.0505, -0.0428, -0.0543,  0.0060,  0.0718,
        -0.0494,  0.0033, -0.0310, -0.0633,  0.0071,  0.0415,  0.0021, -0.0073,
        -0.0649,  0.0008,  0.0810, -0.0247], grad_fn=<SqueezeBackward0>)


## inverser transform of thetas

In [14]:
def transform_intercepts_continous(theta_tilde:torch.Tensor) -> torch.Tensor:
    
    """
    Transforms the unordered theta_tilde to ordered theta values for the bernstein polynomial
    E.G: 
    theta_1 = theta_tilde_1
    theta_2 = theta_tilde_1 + exp(theta_tilde_2)
    ..
    :param theta_tilde: The unordered theta_tilde values
    :return: The ordered theta values
    """

    # Compute the shift based on the last dimension size
    last_dim_size = theta_tilde.shape[-1]
    shift = torch.log(torch.tensor(2.0)) * last_dim_size / 2

    # Get the width values by applying softplus from the second position onward
    widths = torch.nn.functional.softplus(theta_tilde[..., 1:])

    # Concatenate the first value (raw) with the softplus-transformed widths
    widths = torch.cat([theta_tilde[..., [0]], widths], dim=-1)

    # Return the cumulative sum minus the shift
    return torch.cumsum(widths, dim=-1) - shift


In [32]:

def inverse_transform_intercepts_continous(theta: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    """
    Inverse of transform_intercepts_continous.
    Recovers theta_tilde from theta with numerical stability.
    Uses float64 internally, but returns the same dtype as input.
    """
    # Store original dtype and cast to float64 for precision
    orig_dtype = theta.dtype
    theta = theta.to(torch.float64)

    last_dim_size = theta.shape[-1]
    shift = torch.log(torch.tensor(2.0, dtype=torch.float64)) * last_dim_size / 2

    # Undo shift
    theta_shifted = theta + shift

    # Recover widths
    widths = torch.empty_like(theta_shifted)
    widths[..., 0] = theta_shifted[..., 0]
    widths[..., 1:] = theta_shifted[..., 1:] - theta_shifted[..., :-1]

    # Invert softplus (with clamp for numerical stability)
    theta_tilde = torch.empty_like(theta_shifted)
    theta_tilde[..., 0] = widths[..., 0]
    theta_tilde[..., 1:] = torch.log(torch.expm1(torch.clamp(widths[..., 1:], min=eps)))

    return theta_tilde.to(orig_dtype)




 thetas from COLR MODEL in R:

```
library(tram)
weight_init <- read_csv("weight_init.csv")
weight_init$ones <- rep(1, nrow(weight_init))
colr=Colr(x1~ones, data = weight_init,order=19)
colr$weights
colr$theta
```



```
thetas = torch.tensor([
    -2.03300268, -1.43547929, -1.14005127, -1.14005125, -0.13044674,
     0.04618960,  0.04618962,  0.04618963,  0.16484659,  0.16484661,
     0.16484662,  0.16484664,  0.16484665,  0.16484667,  0.16484668,
     0.16484670,  0.16484671,  0.40602064,  1.31082429,  2.38310636
], dtype=torch.float64)
```

In [40]:



thetas = torch.tensor([
    -2.03300268, -1.43547929, -1.14005127, -1.14005125, -0.13044674,
     0.04618960,  0.04618962,  0.04618963,  0.16484659,  0.16484661,
     0.16484662,  0.16484664,  0.16484665,  0.16484667,  0.16484668,
     0.16484670,  0.16484671,  0.40602064,  1.31082429,  2.38310636
], dtype=torch.float64)

theta_tilde_recovered = inverse_transform_intercepts_continous(thetas)


thetas_recovered = transform_intercepts_continous(theta_tilde_recovered)

print("Recovered theta_tilde:\n", theta_tilde_recovered)

print("Recovered thetas_recovered:\n", thetas_recovered)


print("Round-trip error:", torch.max(torch.abs(thetas - thetas_recovered)))


Recovered theta_tilde:
 tensor([  4.8985,  -0.2014,  -1.0680, -17.7275,   0.5565,  -1.6440, -17.7275,
        -18.4207,  -2.0716, -17.7275, -18.4207, -17.7275, -18.4207, -17.7275,
        -18.4207, -17.7275, -18.4207,  -1.2992,   0.3862,   0.6534],
       dtype=torch.float64)
Recovered thetas_recovered:
 tensor([-2.0330, -1.4355, -1.1401, -1.1401, -0.1304,  0.0462,  0.0462,  0.0462,
         0.1648,  0.1648,  0.1648,  0.1648,  0.1648,  0.1648,  0.1648,  0.1648,
         0.1648,  0.4060,  1.3108,  2.3831], dtype=torch.float64)
Round-trip error: tensor(1.9047e-08, dtype=torch.float64)
